In [1]:
%load_ext autoreload
%autoreload 2
%run ~/FKMC/notebooks/notebook_preamble.py
%matplotlib inline

Imported pathlib::Path, os, munch::Munch, pickle, logging, itertools::count, matplotlib::colors
Names are pyplot = plt, matplotlib = mpl, numpy = np


In [2]:
from pathlib import Path
import numpy as np
import os
from time import time, sleep
import sys
from munch import Munch
from itertools import product, islice
from FKMC.montecarlo import *
from FKMC.caching_montecarlo import FK_mcmc as cached_FK_mcmc
from numpy.random import Generator, PCG64

T = 2.25
Ns = [270,]
make_initial_state = lambda N: np.arange(N, dtype = np.float64) % 2 #a CDW initial state
previous_states = [make_initial_state(N) for N in Ns]
J = 5
U = 5
alpha = 1.25


previous_states = [np.array(list(map(float, """0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0.
 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0.
 1. 1. 1. 1. 0. 0.""".split()))), ]

proposal = p_multi_site_uniform_reflect

Ns = [270,]
N_steps = int(10000)
previous_states = previous_states = [make_initial_state(N) for N in Ns]

In [3]:
logs = np.empty(shape = len(Ns), dtype = object)

t1 = time()
for i, N in enumerate(Ns):
    print(f'Starting N = {N}')
    parameters = dict(t = 1, alpha = alpha, mu = 0, beta = 1/T, J = J, U = U, normalise = True)
    MCMC_params = dict(
            state = np.copy(previous_states[i]),
            N_steps = N_steps,
            N_burn_in = 0,
            thin = 1,
            logger = Eigenspectrum_IPR_all(bins = 10000, limit = 20),
            proposal = proposal,
            accept_function = perturbation_accept,
            warnings = True,
            raw_steps = True,
        )

    t0 = time()
    rng = Generator(PCG64(12345))
    logs[i] = FK_mcmc(**MCMC_params, parameters = parameters, rng = rng)
    logs[i].time = time() - t0
    print(f'This N = {N} j ={i} took {time() - t0:.0f} seconds.')
    
logs[0].state[-1]

Starting N = 270
N = 270: 0% through after 0.00m             acceptance rates: classical = 0% quantum = 0% overall = 0%
N = 270: 10% through after 0.08m             acceptance rates: classical = 0.5% quantum = 1e+02% overall = 0.5%
N = 270: 20% through after 0.15m             acceptance rates: classical = 0.6% quantum = 1e+02% overall = 0.6%
N = 270: 30% through after 0.23m             acceptance rates: classical = 0.4% quantum = 1e+02% overall = 0.4%
N = 270: 40% through after 0.30m             acceptance rates: classical = 0.55% quantum = 1e+02% overall = 0.55%
N = 270: 50% through after 0.38m             acceptance rates: classical = 0.56% quantum = 1e+02% overall = 0.56%
N = 270: 60% through after 0.46m             acceptance rates: classical = 0.57% quantum = 1e+02% overall = 0.57%
N = 270: 70% through after 0.54m             acceptance rates: classical = 0.59% quantum = 1e+02% overall = 0.59%
N = 270: 80% through after 0.63m             acceptance rates: classical = 0.53% quantum

/Users/tom/FKMC/FKMC/montecarlo.py:202: RuntimeWarning: underflow encountered in multiply
  self.IPRs[j] = ((evecs * np.conj(evecs))**2).sum(axis = 0)
/Users/tom/FKMC/FKMC/montecarlo.py:202: RuntimeWarning: underflow encountered in square
  self.IPRs[j] = ((evecs * np.conj(evecs))**2).sum(axis = 0)


N = 270: 90% through after 0.71m             acceptance rates: classical = 0.48% quantum = 98% overall = 0.47%
Warning, p_acc = 0.0043, {'t': 1, 'alpha': 1.25, 'mu': 0, 'beta': 0.4444444444444444, 'J': 5, 'U': 5, 'normalise': True, 'J_matrix': 'suppressed for brevity'}
This N = 270 j =0 took 48 seconds.


array([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1.

In [6]:
t1 = time()

for i, N in enumerate(Ns):
    print(f'Starting N = {N}')
    parameters = dict(t = 1, alpha = alpha, mu = 0, beta = 1/T, J = J, U = U, normalise = True)
    MCMC_params = dict(
            state = np.copy(previous_states[i]),
            N_steps = N_steps,
            N_burn_in = int(0),
            thin = 1,
            proposal = proposal,
            warnings = True,
            raw_steps = True,
        )

    t0 = time()
    rng = Generator(PCG64(12345))
    counts, full_cache, last_state, states = cached_FK_mcmc(**MCMC_params, parameters = parameters, rng = rng)

    print(f'This N = {N} j ={i} took {time() - t0:.0f} seconds.')
states[-1]

Starting N = 270
N = 270: 0% through after 0.00m             acceptance rate: classical = 0% quantum = 0% overall = 0%             Ff cache 0 full: 0
N = 270: 10% through after 0.00m             acceptance rate: classical = 0.5% quantum = 1e+02% overall = 0.5%             Ff cache 0.3 full: 50
N = 270: 20% through after 0.00m             acceptance rate: classical = 0.6% quantum = 1e+02% overall = 0.6%             Ff cache 0.5 full: 77
N = 270: 30% through after 0.01m             acceptance rate: classical = 0.4% quantum = 1e+02% overall = 0.4%             Ff cache 0.33 full: 77
N = 270: 40% through after 0.01m             acceptance rate: classical = 0.55% quantum = 1e+02% overall = 0.55%             Ff cache 0.5 full: 87
N = 270: 50% through after 0.01m             acceptance rate: classical = 0.56% quantum = 1e+02% overall = 0.56%             Ff cache 0.52 full: 90
N = 270: 60% through after 0.01m             acceptance rate: classical = 0.57% quantum = 1e+02% overall = 0.57%       

array([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1.

In [5]:
agreement = np.all(logs[0].state == states, axis = 1) 
sum(agreement), agreement.size

(10000, 10000)